In [75]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from dateutil.parser import parse
import datetime
os.getcwd()

'/Linux/[2021, 2022] 초분광(Hyperspectral) 데이터/[ 2022 ]/PSC'

# **Dataset preprocessing**

In [76]:
def data_set (data) :
    day_df = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Start Min", "End Min"] + list(data.columns))

    for i in tqdm(range(data.shape[0])) : 
        obs = data.iloc[[i], :]                                         # obs : 데이터 하나
        time = parse(obs["timestamp"].values[0])                        # obs의 time 정보
        time = time + datetime.timedelta(hours = 9)

        # start = 0이면 해당 날짜의 데이터 수집 시작 알림
        if (time.hour == 0) & (time.minute < 10) :
            print("{}년 {}월 {}일 데이터 수집 시작".format(time.year, time.month, time.day))
        
        # obs의 관측 time(hour)이 하루가 지나면 데이터 수집 종료
        elif (time.hour == 23) & (time.minute > 50) :
            print("{}년 {}월 {}일 데이터 수집 종료".format(time.year, time.month, time.day))

        # obs의 관측 time(hour)이 0~23이면 데이터 수집
        if time.hour in range(0, 24) :
            Y, M, D, H, Min = time.year, time.month, time.day, time.hour, time.minute

            if time.minute in range(0, 15+1) :
                inform = pd.DataFrame([Y, M, D, H, 0, 15] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(15, 30+1) :
                inform = pd.DataFrame([Y, M, D, H, 15, 30] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(30, 45+1) :
                inform = pd.DataFrame([Y, M, D, H, 30, 45] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)

            elif time.minute in range(45, 60+1) :
                inform = pd.DataFrame([Y, M, D, H, 45, 0] + list(obs.values[0]), index = day_df.columns).T
                day_df = pd.concat([day_df, inform], axis = 0)
        

    day_df = day_df.drop(columns = ["timestamp"])
    complete_data = day_df.groupby(["Year", "Month", "Day", "Hour", "Start Min", "End Min"]).sum()
    
    return complete_data

In [77]:
def data_set2 (data) :
    day_df = pd.DataFrame(columns = ["Year", "Month", "Day", "Hour", "Min"] + list(data.columns))

    for i in tqdm(range(data.shape[0])) : 
        obs = data.iloc[[i], :]                                         # obs : 데이터 하나
        time = parse(obs["timestamp"].values[0])                        # obs의 time 정보
        time = time + datetime.timedelta(hours = 9)

        Y, M, D, H, Min = time.year, time.month, time.day, time.hour, time.minute
        
        if time.minute in range(0, 15) :
            inform = pd.DataFrame([Y, M, D, H, 15] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(15, 30) :
            inform = pd.DataFrame([Y, M, D, H, 30] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(30, 45) :
            inform = pd.DataFrame([Y, M, D, H, 45] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)

        elif time.minute in range(45, 60) :
            time2 = time + datetime.timedelta(minutes = 15)
            Y_, M_, D_, H_= time2.year, time2.month, time2.day, time2.hour
            inform = pd.DataFrame([Y_, M_, D_, H_, 0] + list(obs.values[0]), index = day_df.columns).T
            day_df = pd.concat([day_df, inform], axis = 0)


    day_df = day_df.drop(columns = ["timestamp"]).reset_index(drop = True)
        
    return day_df

In [78]:
data = pd.read_csv("../DAT/Haman.csv")
print(data.shape)
data.head()

(19740, 569)


,ct,reflectance.000,reflectance.001,reflectance.002,reflectance.003,reflectance.004,reflectance.005,reflectance.006,reflectance.007,reflectance.008,...,ph,turbidity,chlorophyll,phycocyanin,waterDepth,airTemperature,humidity,windSpeed,windDirection,insolation
0,20220418T072421,0.004747,0.004863,0.004779,0.004673,0.004558,0.004505,0.004428,0.004447,0.004497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220418T073920,0.004034,0.004189,0.004177,0.004113,0.004002,0.003988,0.003913,0.003917,0.003981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220418T075420,0.005296,0.005435,0.005381,0.005250,0.005133,0.005090,0.005072,0.005152,0.005197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220418T080920,0.005550,0.005703,0.005649,0.005581,0.005449,0.005434,0.005392,0.005452,0.005534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220418T082420,0.005321,0.005448,0.005385,0.005255,0.005162,0.005182,0.005135,0.005186,0.005233,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
data_519 = data.iloc[5928:,:].reset_index(drop = True)
data_519.rename({"ct" : "timestamp"}, axis = 1, inplace = True)
data_519.head()

,timestamp,reflectance.000,reflectance.001,reflectance.002,reflectance.003,reflectance.004,reflectance.005,reflectance.006,reflectance.007,reflectance.008,...,ph,turbidity,chlorophyll,phycocyanin,waterDepth,airTemperature,humidity,windSpeed,windDirection,insolation
0,20220519T000925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.815,2.13,7.30,0.15,0.08,NaN,NaN,NaN,NaN,NaN
1,20220519T000931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20.5,50.1,5.1,1.3,0.0
2,20220519T002424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.826,2.28,8.02,0.15,0.08,NaN,NaN,NaN,NaN,NaN
3,20220519T002430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.4,50.1,357.9,1.0,0.0
4,20220519T003924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.836,2.20,7.58,0.15,0.08,NaN,NaN,NaN,NaN,NaN


In [80]:
data_519.columns[-20:]

Index(['reflectance.548', 'reflectance.549', 'reflectance.550', 'chla', 'tsm',
       'kd', 'cpc', 'waterTemperature', 'conductivity', 'do', 'ph',
       'turbidity', 'chlorophyll', 'phycocyanin', 'waterDepth',
       'airTemperature', 'humidity', 'windSpeed', 'windDirection',
       'insolation'],
      dtype='object')

In [81]:
water = data_519[["timestamp", 'chlorophyll', 'conductivity', 'do', 'ph', 'phycocyanin', 'turbidity', 'waterDepth', 'waterTemperature']].dropna().reset_index(drop = True)
weather = data_519[["timestamp", 'airTemperature', 'humidity', 'insolation', 'windDirection', 'windSpeed']].dropna().reset_index(drop = True)
hs = data_519[["timestamp", "chla"] + list(data.filter(regex = 'reflectance').columns)].dropna().reset_index(drop = True)

print(water.shape, weather.shape, hs.shape)

(6624, 9) (6623, 6) (563, 553)


In [82]:
new_water = data_set2(water)
new_weather = data_set2(weather)
new_hs = data_set2(hs)

  0%|          | 0/6624 [00:00<?, ?it/s]

  0%|          | 0/6623 [00:00<?, ?it/s]

  0%|          | 0/563 [00:00<?, ?it/s]

In [83]:
display(new_water.head(3))
display(new_weather.head(3))
display(new_hs.head(3))

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,turbidity,waterDepth,waterTemperature
0,2022,5,19,9,15,7.3,397.74,12.71,8.815,0.15,2.13,0.08,22.01
1,2022,5,19,9,30,8.02,397.69,12.75,8.826,0.15,2.28,0.08,22.06
2,2022,5,19,9,45,7.58,397.89,12.84,8.836,0.15,2.2,0.08,22.13


,Year,Month,Day,Hour,Min,airTemperature,humidity,insolation,windDirection,windSpeed
0,2022,5,19,9,15,20.5,50.1,0,1.3,5.1
1,2022,5,19,9,30,21.4,50.1,0,1,357.9
2,2022,5,19,9,45,21.9,42.1,0,0.8,0.3


,Year,Month,Day,Hour,Min,chla,reflectance.000,reflectance.001,reflectance.002,reflectance.003,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
0,2022,5,23,10,0,22.1,0.0053611,0.00539296,0.00531906,0.00520581,...,0.00185233,0.00184345,0.00183897,0.00180088,0.00176648,0.00174308,0.00170797,0.00167395,0.0016737,0.00166108
1,2022,5,23,10,15,21.1,0.00517481,0.00522698,0.00516434,0.00507374,...,0.00170406,0.00170097,0.00167255,0.0016582,0.001647,0.00166814,0.00164085,0.00161905,0.00161375,0.00159016
2,2022,5,23,10,30,20.5,0.00507721,0.00509964,0.00504614,0.00496383,...,0.00175763,0.00175349,0.00174779,0.00168539,0.00166061,0.00162415,0.00161178,0.00156037,0.00154746,0.00154148


In [84]:
new_data = pd.merge(left = new_water, right = new_weather, how = 'outer', on = ["Year", "Month", "Day", "Hour", "Min"], sort = True)
new_data = pd.merge(left = new_data, right = new_hs, how = 'outer', on = ["Year", "Month", "Day", "Hour", "Min"], sort = True)  
new_data = new_data.astype({"Year" : "int", "Month" : "int", "Day" : "int", "Hour" : "int", "Min" : "int"})
new_data.head(3)

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
0,2022,5,19,9,15,7.3,397.74,12.71,8.815,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,5,19,9,30,8.02,397.69,12.75,8.826,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,5,19,9,45,7.58,397.89,12.84,8.836,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
new_data.tail(5)

,Year,Month,Day,Hour,Min,chlorophyll,conductivity,do,ph,phycocyanin,...,reflectance.541,reflectance.542,reflectance.543,reflectance.544,reflectance.545,reflectance.546,reflectance.547,reflectance.548,reflectance.549,reflectance.550
6620,2022,7,27,8,45,12.05,236.37,9.96,8.974,2.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6621,2022,7,27,9,0,11.24,233.42,10,8.974,6.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6622,2022,7,27,9,15,13.36,230.16,10.2,9.013,9.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6623,2022,7,27,9,30,10.02,229.21,10.56,9.096,1.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6624,2022,7,27,9,45,13.02,230.02,10.6,9.05,2.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# new_data.to_csv("../DAT/complete_data(haman-chla).csv", index = False)